## Kemal Demirel: 191104091

## Kadir Çağan Eren: 

In [1]:
import glob
import pathlib
import cv2
import numpy as np 
import pandas as pd 
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


## Read all dataset with images

In [32]:
def traverse_directory(path, isItTrain):
    entered = False
    dataset = []
    labels = []
    for dirpath, dirnames, filenames in os.walk(path):
        if not entered:
            entered = True    
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            if(file_path.endswith(".jpg")):
                img = cv2.imread(file_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (64, 64))
                dataset.append(img)
                tmp_path = dirpath
                if(isItTrain):
                    label = tmp_path[tmp_path.find("train/")+len("train/"):]    
                else:
                    label = tmp_path[tmp_path.find("test/")+len("test/"):]    
                labels.append(label)
    return np.array(dataset), np.array(labels)

In [34]:
X_train, y_train = traverse_directory(os.getcwd()+"/dataset/train", True)

In [35]:
X_test, y_test = traverse_directory(os.getcwd()+"/dataset/test", False)

In [36]:
def extract_features_histogram(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist)
    return hist.flatten()

In [ ]:
def extract_features_sift(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Initialize a SIFT object and detect keypoints and descriptors
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)

    # Return the descriptors as a 1D vector
    return descriptors.flatten()

In [37]:
X_train_features = np.array([extract_features_histogram(image) for image in X_train])
X_test_features = np.array([extract_features_histogram(image) for image in X_test])

In [38]:
# Train a classifier
def train_classifier(X_train, y_train):
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train)
    return svm

In [39]:
def evaluate_classifier(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return accuracy, precision, recall, f1

In [40]:

# Predict new images
def predict(classifier, image):
    feature = extract_features_histogram(image)
    prediction = classifier.predict([feature])
    return prediction[0]

In [41]:
classifier = train_classifier(X_train_features, y_train)

In [42]:
accuracy, precision, recall, f1 = evaluate_classifier(classifier, X_test_features, y_test)

# Predict new images
new_image = cv2.imread("/Users/kemaldemirel/Desktop/Lectures/Bil 468/Project/Computer-Vision-Fruit-Vegetables-Detection/dataset/test/grapes/Image_1.jpg")
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (64, 64))
prediction = predict(classifier, new_image)
print(precision)

0.5350395646474078


/Users/kemaldemirel/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
